In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima  # Auto-tuning ARIMA parameters

from modules.utils import feature_engineering_last, filter_data_by_year_month

In [3]:
X = pd.read_csv(
   "/Users/elouan/A. Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT"
)

In [4]:
YEARS = [2020, 2021, 2022, 2023, 2024, 2025]

Data = filter_data_by_year_month(X, YEARS)


------------------------------------------------
Some graphs on the return 

In [ ]:
Data["return"] = np.log(Data["close"] / Data["close"].shift(1))


In [24]:
df["log_return"] = np.log(df["close"] / df["close"].shift(1))

y = df["log_return"]

y = y.iloc[1:].reset_index(drop=True)

/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [25]:
y.head()

0    0.000353
1    0.000243
2   -0.000265
3    0.000060
4   -0.000130
Name: log_return, dtype: float64

In [29]:
y = y.dropna()  # or df = df.fillna(0)

In [30]:
import numpy as np

# Check for NaN values in the DataFrame
print(y.isna().sum())  # Counts the NaNs for each column

# Check for infinite values
print(np.isinf(y).sum())  # Counts the infinities for each column

0
0


In [31]:
from statsmodels.tsa.stattools import adfuller

# Perform Augmented Dickey-Fuller Test to know is the series is stationnary or not
result = adfuller(y)

print("ADF Statistic:", result[0])
print("p-value:", result[1])
if result[1] < 0.05:
    print("✅ Data is stationary (p < 0.05)")
else:
    print("❌ Data is NOT stationary (p > 0.05) - Differencing needed")

ADF Statistic: -58.42625035941062
p-value: 0.0
✅ Data is stationary (p < 0.05)


In [32]:
stepwise_fit = auto_arima(y, seasonal=False, trace=True, suppress_warnings=True)
print(stepwise_fit.summary())

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-523607.042, Time=31.93 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-515599.654, Time=4.72 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-523015.190, Time=4.16 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-523544.623, Time=5.68 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=-523875.888, Time=24.27 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=-523603.211, Time=7.74 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-523607.885, Time=18.67 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=-523607.772, Time=12.76 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=-523614.333, Time=10.27 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-523610.549, Time=9.11 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=-523629.841, Time=15.87 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-523873.947, Time=77.30 sec

Best model:  ARIMA(1,0,2)(0,0,0)[0]          
Total fit time: 222.678 seconds
                               SARIMAX Resu

In [33]:
# Use best (p,d,q) from auto_arima()
p, d, q = stepwise_fit.order

# Fit ARIMA model on log returns
model = ARIMA(y, order=(p, d, q))
model_fit = model.fit()

# Print model summary
print(model_fit.summary())

/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:             log_return   No. Observations:               213354
Model:                 ARIMA(1, 0, 2)   Log Likelihood              261941.970
Date:                Mon, 10 Mar 2025   AIC                        -523873.940
Time:                        16:34:40   BIC                        -523822.586
Sample:                             0   HQIC                       -523858.868
                             - 213354                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.619e-05      0.000     -0.292      0.770      -0.000       0.000
ar.L1         -0.8732      0.001   -682.037      0.000      -0.876      -0.871
ma.L1          0.6798      0.001    526.421      0.0

In [34]:
forecast_steps = 5  # 40 x 15-minute = 10 hours

# Forecast log returns
forecast_log_return = model_fit.forecast(steps=forecast_steps)
print(forecast_log_return.head())  # Check first few values

213354    1.851581e-04
213355   -8.969853e-05
213356    1.054065e-05
213357   -7.698569e-05
213358   -5.598837e-07
Name: predicted_mean, dtype: float64


/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [40]:
from sklearn.metrics import accuracy_score

# Define the training size (number of historical points to train on)
train_size = int(len(y) * 0.8)  # Use 80% of data for training


def convert_to_binary_labels(log_returns):
    return (log_returns > 0).astype(int)


# Convert the actual log returns into binary labels
actual_log_returns = y.iloc[train_size + 1 :].values
actual_labels = convert_to_binary_labels(actual_log_returns)

# Backtesting loop: rolling window (with binary classification)
predictions_binary = []


# Backtesting loop: rolling window
for i in range(train_size, train_size + 10):
    train_data = y.iloc[:i]  # Train on the first `i` data points
    test_data = y.iloc[i : i + 1]  # The next log return to predict

    # Fit the model on the current training data
    model = ARIMA(train_data, order=(p, d, q))
    model_fit = model.fit()

    # Make a prediction for the next step (log return)
    forecast_log_return = model_fit.forecast(steps=1)

    # Store the predicted log return

    prediction_binary = convert_to_binary_labels(forecast_log_return)
    predictions_binary.append(prediction_binary)


# Compare predicted binary labels to actual binary labels
accuracy = accuracy_score(actual_labels[:10], predictions_binary)
print(f"Accuracy: {accuracy:.4f}")

/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/elouan/anaconda3/envs/Predicting_crypto/li

Accuracy: 0.2000


/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


_________________
GARCH

In [ ]:
X["log_return"] = np.log(X["close"] / X["close"].shift(1))
X.dropna(inplace=True)

In [ ]:
from arch import arch_model

# Fit GARCH on log returns
garch = arch_model(X["log_return"].dropna(), vol="Garch", p=1, q=1)
garch_fit = garch.fit(disp="off")

# Forecast volatility (next 30 steps)
forecast_vol = garch_fit.forecast(start=len(df), horizon=30)